In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

C:\Users\ADMIN\anaconda3\envs\datascience\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [32]:
tokens = tokenizer.encode('I am not good', return_tensors='pt')

In [33]:
tokens

tensor([[  101,   151, 10345, 10497, 12050,   102]])

In [34]:
tokenizer.decode(tokens[0])

'[CLS] i am not good [SEP]'

In [35]:
result = model(tokens)

In [36]:
torch.argmax(result.logits)

tensor(0)

In [37]:
result.logits

tensor([[ 2.7766,  2.5987,  1.0689, -2.0894, -3.5393]],
       grad_fn=<AddmmBackward0>)

In [38]:
int(torch.argmax(result.logits))+1

1

In [12]:
r= requests.get('https://www.yelp.com/biz/cafe-broadway-san-francisco')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [13]:
results[0].text

'Service was standard- you order by marking on an erasable menu. Restaurant itself was clean and food came out quickly good for small groups - probably nothing over a party of 6. Food was surprisingly very good and reasonable price. The lava egg yolk buns were a tad understeamed so minus 1 star but otherwise all the food was amazing. Some stand outs were the crispy shrimp rice noodle (Cheung fun), fried meat taro balls, shiu mai and bbq steak bao. Overall would definitely come back!'

In [14]:
reviews[0]

'Service was standard- you order by marking on an erasable menu. Restaurant itself was clean and food came out quickly good for small groups - probably nothing over a party of 6. Food was surprisingly very good and reasonable price. The lava egg yolk buns were a tad understeamed so minus 1 star but otherwise all the food was amazing. Some stand outs were the crispy shrimp rice noodle (Cheung fun), fried meat taro balls, shiu mai and bbq steak bao. Overall would definitely come back!'

In [15]:
import numpy as np
import pandas as pd

In [16]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [17]:
df.tail()

,review
5,Solid 4 stars. My boyfriend and I came here fo...
6,Average San Francisco has a few Chinatowns. Th...
7,Unassuming spot in the heart of Chinatown. We ...
8,Actually open during Chinese new year in 2023....
9,Family liked shrimp dumpling and lunch special...


In [18]:
df['review'].iloc[0]

'Service was standard- you order by marking on an erasable menu. Restaurant itself was clean and food came out quickly good for small groups - probably nothing over a party of 6. Food was surprisingly very good and reasonable price. The lava egg yolk buns were a tad understeamed so minus 1 star but otherwise all the food was amazing. Some stand outs were the crispy shrimp rice noodle (Cheung fun), fried meat taro balls, shiu mai and bbq steak bao. Overall would definitely come back!'

In [39]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [40]:
sentiment_score(df['review'].iloc[0])

4

In [41]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [42]:
df['review']

0    Service was standard- you order by marking on ...
1    We stayed in Chinatown and were right across t...
2    This place is so cute ! And inexpensive ! Real...
3    Why are they singing karaoke in the MIDDLE OF ...
4    I was running to meet some friends for a North...
5    Solid 4 stars. My boyfriend and I came here fo...
6    Average San Francisco has a few Chinatowns. Th...
7    Unassuming spot in the heart of Chinatown. We ...
8    Actually open during Chinese new year in 2023....
9    Family liked shrimp dumpling and lunch special...
Name: review, dtype: object

In [43]:
df

,review,sentiment
0,Service was standard- you order by marking on ...,4
1,We stayed in Chinatown and were right across t...,4
2,This place is so cute ! And inexpensive ! Real...,5
3,Why are they singing karaoke in the MIDDLE OF ...,1
4,I was running to meet some friends for a North...,4
5,Solid 4 stars. My boyfriend and I came here fo...,4
6,Average San Francisco has a few Chinatowns. Th...,3
7,Unassuming spot in the heart of Chinatown. We ...,2
8,Actually open during Chinese new year in 2023....,5
9,Family liked shrimp dumpling and lunch special...,4


In [44]:
df['review'].iloc[2]

'This place is so cute ! And inexpensive ! Real HK food and great service! Get the Peking duck'